# Full outcome model

In [12]:
# import required modules
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats

from classes.pathway import SSNAP_Pathway
from classes.clinical_outcome import Clinical_outcome

## Import data

In [14]:
hospital_performance = pd.read_csv('data/hospital_performance_thrombectomy.csv')

In [15]:
mrs_dists = pd.read_csv('data/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

In [16]:
mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


In [17]:
hospital_performance.head(5).T

,0,1,2,3,4
stroke_team,Addenbrooke's Hospital,Basildon University Hospital,Blackpool Victoria Hospital,Broomfield Hospital,Calderdale Royal Hospital
admissions,602.166667,486.5,485.833333,452.166667,634.666667
proportion_of_all_with_ivt,0.149184,0.132237,0.091938,0.104681,0.135504
proportion_of_all_with_mt,0.026571,0.01199,0.01235,0.006635,0.00604
proportion_of_mt_with_ivt,0.520833,0.6,0.444444,0.555556,0.73913
proportion1_of_all_with_onset_known_ivt,0.590645,0.648167,0.44837,0.558791,0.546218
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.57241,0.693191,0.605541,0.645673
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.958449,0.941501,0.991285,0.946389
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.888247,0.888628,0.915385,0.929976
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0


# Run trials

## Same hospital, multiple trials

In [22]:
# Set general model parameters
trials = 4

# Get data for one hospital
hospital_name = hospital_performance.iloc[0]['stroke_team']
hospital_data = hospital_performance.iloc[0]

patient_array = SSNAP_Pathway(hospital_name, hospital_data)
for trial in range(trials):
    patient_array.run_trial()

Show the results for the first few patients in the latest trial:

In [28]:
patient_array.results_dataframe.head(10).T

,0,1,2,3,4,5,6,7,8,9
onset_time_known_bool,True,False,False,False,True,True,True,False,True,True
onset_to_arrival_mins,121.0,NaN,NaN,NaN,350.0,477.0,481.0,NaN,64.0,187.0
onset_to_arrival_on_time_ivt_bool,True,False,False,False,False,False,False,False,True,True
onset_to_arrival_on_time_mt_bool,True,False,False,False,True,True,False,False,True,True
arrival_to_scan_mins,19.0,128.0,63.0,77.0,49.0,78.0,45.0,105.0,71.0,43.0
arrival_to_scan_on_time_ivt_bool,True,True,True,True,True,True,True,True,True,True
arrival_to_scan_on_time_mt_bool,True,True,True,True,True,True,True,True,True,True
onset_to_scan_mins,140.0,NaN,NaN,NaN,399.0,555.0,526.0,NaN,135.0,230.0
onset_to_scan_on_time_ivt_bool,True,False,False,False,False,False,False,False,True,True
time_left_for_ivt_after_scan_mins,130.0,NaN,NaN,NaN,-0.0,-0.0,-0.0,NaN,135.0,40.0


Show just the arrival to scan times for each patient in the latest trial:

In [29]:
patient_array.trial['arrival_to_scan_mins'].data

array([ 19., 128.,  63.,  77.,  49.,  78.,  45., 105.,  71.,  43.,  47.,
         5., 414.,  68.,  46.,  48.,  67.,  72.,  23.,   9.,  45.,  60.,
        24., 481.,  11., 172.,   7.,  20., 270., 132.,  26.,  51.,  68.,
        21.,  98.,  42.,  23.,  17.,  36., 101.,  31.,  36.,  18.,  57.,
        80., 116.,   4.,  83.,  23., 364.,  10.,  16.,  70.,  20.,  31.,
        44.,  52.,  47.,  72., 591.,  34.,  25.,  30.,  70.,  29.,  29.,
        73.,  62.,  75.,  98.,  46.,  22.,  62.,  28.,  18.,  46.,  23.,
       481.,  19.,  91.,  37.,  69.,  95.,  54.,  13.,  44.,  89.,  40.,
        57.,  69.,  45., 104.,  35.,  25.,  19.,  65.,  23.,  48.,  82.,
        61.,  14.,  40.,  29.,  60.,  54.,  25.,  11.,  24.,  43., 179.,
        72.,  43.,  35.,  30.,  38.,   6., 131.,  82.,  63., 354.,  39.,
        30.,  12., 207.,  52.,  93.,  76.,  13.,  39.,  97., 106.,  52.,
        20.,  28.,  64.,  20.,  62.,  22.,  73., 113., 168.,  21.,  22.,
        28.,  42.,  21.,  52.,  27.,  24.,  39.,  3

Show the hospital performance of each trial:

In [24]:
patient_array.df_performance

,Target,Trial_1,Trial_2,Trial_3,Trial_4
stroke_team,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital,Addenbrooke's Hospital
admissions,602.166667,602,602,602,602
proportion_of_all_with_ivt,0.149184,0.166113,0.17608,0.137874,0.124585
proportion_of_all_with_mt,0.026571,0.021595,0.021595,0.019934,0.021595
proportion_of_mt_with_ivt,0.520833,0.461538,0.461538,0.5,0.461538
proportion1_of_all_with_onset_known_ivt,0.590645,0.60299,0.609635,0.55814,0.586379
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt,0.679007,0.688705,0.716621,0.681548,0.70255
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt,0.94893,0.984,0.980989,0.982533,0.983871
proportion4_of_mask3_with_onset_to_scan_on_time_ivt,0.845818,0.861789,0.906977,0.84,0.856557
proportion5_of_mask4_with_enough_time_to_treat_ivt,1.0,1.0,1.0,1.0,1.0


## Same hospital, multiple scenarios

not implemented yet, but will involve overwriting the hospital performance data similarly to the following cell:

In [30]:
# Scenario changes to the performance data
# Create a fresh copy of the original performance data
hospital_performance_series = hospital_performance.iloc[0].copy()

# Speed scenario
# All patients are scanned within 4hrs of arrival
# (how does this work for the masks picking different times for MT and IVT? Someteims 4hr, sometimes 8hr)
hospital_performance_series['proportion3_of_mask2_with_arrival_to_scan_on_time_ivt'] = 1.0

# Onset time scenario
# More patients have their onset time determined
hospital_performance_series['proportion1_of_all_with_onset_known_ivt'] = 1.0

# Benchmark scenario
# The proportion of eligible patients receiving treatment is 
# in line with the benchmark teams' proportions.
hospital_performance_series['proportion6_of_mask5_with_treated_ivt'] = 1.0

print(hospital_performance_series)


stroke_team                                               Addenbrooke's Hospital
admissions                                                            602.166667
proportion_of_all_with_ivt                                              0.149184
proportion_of_all_with_mt                                               0.026571
proportion_of_mt_with_ivt                                               0.520833
proportion1_of_all_with_onset_known_ivt                                      1.0
proportion2_of_mask1_with_onset_to_arrival_on_time_ivt                  0.679007
proportion3_of_mask2_with_arrival_to_scan_on_time_ivt                        1.0
proportion4_of_mask3_with_onset_to_scan_on_time_ivt                     0.845818
proportion5_of_mask4_with_enough_time_to_treat_ivt                           1.0
proportion6_of_mask5_with_treated_ivt                                        1.0
lognorm_mu_onset_arrival_mins_ivt                                       4.636923
lognorm_sigma_onset_arrival_

## All hospitals, multiple trials and outcomes

This needs tidying up

In [ ]:

# Set general model parameters
trials = 100

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    'onset_to_needle_mins',
    'onset_to_puncture_mins'
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = []
for column in outcome_results_columns:
    columns_here = [column + ending for ending in results_types]
    results_columns += columns_here
# Also store onset to needle time:
# results_columns += ['Onset_to_needle_(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'Thrombectomy_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    'onset_to_needle_mins',
    'onset_to_puncture_mins'
    ]

# Iterate through hospitals
scenario_counter = 0
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[1]['stroke_team']
    hospital_data = hospital[1]

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    number_of_patients = int(run_data['admissions'])

    patient_array = SSNAP_Pathway(hospital_name, hospital_data)
    for trial in range(trials):

        patient_array.run_trial()
        
        # Initiate the outcome model object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        for key in clinical_outcome.trial.keys():
            if key in patient_array.trial.keys():
                clinical_outcome.trial[key].data = patient_array.trial[key].data
        
        # Calculate outcomes:
        results_by_stroke_type, patient_array_outcomes = clinical_outcome.calculate_outcomes()
        
#         # mask = (
#         #     patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
#         # onset_to_needle_mins_masked = \
#         #         patient_array.patient_array_onset_to_needle_mins[mask].mean()

        # Mean treatment times:
        # (if/else to prevent mean of empty slice RunTime warning)
        onset_to_needle_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_needle_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_needle_mins'].data)) == False
            else np.NaN)
        onset_to_puncture_mins_mean = (
            np.nanmean(patient_array.trial['onset_to_puncture_mins'].data) 
            if np.all(np.isnan(patient_array.trial['onset_to_puncture_mins'].data)) == False
            else np.NaN)
    
        # Save scenario results to dataframe
        result = [
            np.mean(patient_array.trial['ivt_chosen_bool'].data)*100.0,
            np.mean(patient_array.trial['mt_chosen_bool'].data)*100.0,
            results_by_stroke_type['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results_by_stroke_type['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            onset_to_needle_mins_mean,
            onset_to_puncture_mins_mean
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        
        # print(stop, here, please)
        
    trial_result = []
    
    # sometimes these medians etc. are calculated when there's only one or two valid values in the column. Should probably do something about that.
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    # trial_result += [trial_df['onset_to_needle'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result
    

# # Apply calibration
# results_df['calibration'] = calibration
# for col in list(results_df):
#     if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
#         results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


n.b. runtime warning about multiply error a * scale + loc is something to do with the above cell, not either of the pathway or outcome classes it's calling. Maybe that stats.norm.interval() line at the end?

In [ ]:
results_df.head(10).T

In [ ]:
results_df.to_csv('full_outcome_pathway_results_refactored.csv')

Print some info about the final trial:

In [ ]:
print(clinical_outcome)